# Multimodal Early Fusion

## Requirements

Multimodal: Given a query, these retrieval systems selects the tracks that are most similar to the query track, based on more
than one complementary feature (i.e., of different modalities). For this purpose, you will use fusion strategies based on
* Late fusion
*  Early fusion

You will hence implement two different multimodal retrieval systems

In [ ]:
import pandas as pd
import torch
import os, sys
from tqdm import tqdm

target_folder = 'MMSR25-26-Group-E'
current_path = os.getcwd()

while os.path.basename(current_path) != target_folder:
    if os.path.basename(current_path) == 'RetrievalAlgorithm':
        if os.path.join(current_path) not in sys.path:
            sys.path.append(os.path.join(current_path))
    parent = os.path.dirname(current_path)
    os.chdir(parent)
    current_path = parent

from RetrievalAlgorithm.src.utils.data_loading import load_all_tsv_files_from_path
from RetrievalAlgorithm.src.score_calculation_modules.cosine_similarity_module import CosineSimilarityModule
from RetrievalAlgorithm.src.multimodal_early_fusion_calculations import calculate_multimodal_similarity

## Module initialization

In [ ]:
unimodal_module_path = 'RetrievalAlgorithm/modules/cosine_similarity_module.pt'

if os.path.exists(unimodal_module_path):
    unimodal_module = torch.jit.load(unimodal_module_path)
else:
    unimodal_module = CosineSimilarityModule()
    unimodal_module = torch.jit.script(unimodal_module)
    unimodal_module.save(unimodal_module_path)

## Data Loading

In [ ]:
dataset_files_dict = load_all_tsv_files_from_path(path_to_dataset='Dataset')

## Calculating Multimodal Similarity Scores

In [ ]:
from RetrievalAlgorithm.src.normalization import *


norm_names_types_list = [
    ('raw', NormalizationModule),
    ('min_max', MinMaxNormalizationModule),
    ('max_abs', MaxAbsNormalizationModule),
    ('standard', StandardNormalizationModule),
    ('robust', RobustNormalizationModule),
]
sim_scores_df_list = []

### Lyrics + Audio

In [ ]:
for norm_name, norm_module_type in norm_names_types_list:
    print('=' * 100)
    print('Normalization name:', norm_name)

    datasets_list = [dataset_files_dict['id_lyrics_bert_mmsr.tsv'],
                     dataset_files_dict['id_mfcc_bow_mmsr.tsv']]

    lyrics_audio_similarity_scores_df = calculate_multimodal_similarity(
        datasets_df=datasets_list,
        calculation_module=unimodal_module,
        normalization_module_type=norm_module_type,
        feature_name='Lyrics_Audio',
        batch_size=1024,
        include_reverse_pairs=False,
        include_self_pairs=True,
    )

    sim_scores_df_list.append((norm_name, 'lyrics_audio', lyrics_audio_similarity_scores_df))

### Lyrics + Videoclips

In [ ]:
for norm_name, norm_module_type in norm_names_types_list:
    print('=' * 100)
    print('Normalization name:', norm_name)

    datasets_list = [dataset_files_dict['id_lyrics_bert_mmsr.tsv'],
                     dataset_files_dict['id_vgg19_mmsr.tsv']]

    lyrics_videoclips_similarity_scores_df = calculate_multimodal_similarity(
        datasets_df=datasets_list,
        calculation_module=unimodal_module,
        feature_name='Lyrics_Videoclips',
        batch_size=1024,
        include_reverse_pairs=False,
        include_self_pairs=True,
    )

    sim_scores_df_list.append((norm_name, 'lyrics_videoclips', lyrics_videoclips_similarity_scores_df))

### Audio + Videoclips

In [ ]:
for norm_name, norm_module_type in norm_names_types_list:
    print('=' * 100)
    print('Normalization name:', norm_name)

    datasets_list = [dataset_files_dict['id_mfcc_bow_mmsr.tsv'],
                     dataset_files_dict['id_vgg19_mmsr.tsv']]

    audio_videoclips_similarity_scores_df = calculate_multimodal_similarity(
        datasets_df=datasets_list,
        calculation_module=unimodal_module,
        normalization_module_type=norm_module_type,
        feature_name='Audio_Videoclips',
        batch_size=1024,
        include_reverse_pairs=False,
        include_self_pairs=True,
    )

    sim_scores_df_list.append((norm_name, 'audio_videoclips', audio_videoclips_similarity_scores_df))

### Lyrics + Audio + Videoclips

In [ ]:
for norm_name, norm_module_type in norm_names_types_list:
    print('='*100)
    print('Normalization name:', norm_name)

    datasets_list = [dataset_files_dict['id_lyrics_bert_mmsr.tsv'],
                     dataset_files_dict['id_mfcc_bow_mmsr.tsv'],
                     dataset_files_dict['id_vgg19_mmsr.tsv']]

    lyrics_audio_videoclips_similarity_scores_df = calculate_multimodal_similarity(
        datasets_df=datasets_list,
        calculation_module=unimodal_module,
        normalization_module_type=norm_module_type,
        feature_name='Lyrics_Audio_Videoclips',
        batch_size=1024,
        include_reverse_pairs=False,
        include_self_pairs=True,
    )

    sim_scores_df_list.append((norm_name, 'lyrics_audio_videoclips', lyrics_audio_videoclips_similarity_scores_df))

## Merge and Save results

In [ ]:
target_dir = 'RetrievalAlgorithm/results/multimodal/early_fusion'
os.makedirs(target_dir, exist_ok=True)

for norm_name, feature_name, sim_scores_df in tqdm(sim_scores_df_list, desc='Saving similarity scores'):
    output_path = os.path.join(target_dir, norm_name)
    os.makedirs(output_path, exist_ok=True)
    output_path = os.path.join(output_path, f'unimodal{norm_name}_{feature_name}_similarity_scores.parquet')
    sim_scores_df.to_parquet(output_path, index=False)